In [1]:
import sys, os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from sklearn.linear_model import LinearRegression

In [2]:
data_path = Path("G:\My Drive\Colab Notebooks\Zillow\data")

if (not 'df' in locals()) and data_path.joinpath('dfall_logerr.pkl').is_file():
    print('load cache')  
    df = pd.read_pickle(data_path.joinpath('dfall_logerr.pkl')) 
elif (not 'df' in locals()) or (not data_path.joinpath('dfall_logerr.csv').is_file()):
    df = pd.read_pickle(data_path.joinpath('dfall_logerr.pkl'))
    df.to_csv(data_path.joinpath('dfall_logerr.csv'), index = False)
    print('export to csv')


load cache


In [3]:
def printunique(df):
    for colname in df.columns:
        col_tmp = df[colname]
        print(colname)
        print(col_tmp.unique())
    return

In [13]:
df_2016 = df[df['year_data'] == 2016]
df_2016.reset_index(inplace=True, drop=True)
df_2016.info()
printunique(df_2016)
display(df_2016)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      90275 non-null  string 
 1   airconditioningtypeid         90275 non-null  string 
 2   architecturalstyletypeid      90275 non-null  string 
 3   basementsqft                  90275 non-null  int64  
 4   bathroomcnt                   90275 non-null  float64
 5   bedroomcnt                    90275 non-null  int64  
 6   buildingclasstypeid           90275 non-null  string 
 7   buildingqualitytypeid         90275 non-null  string 
 8   calculatedbathnbr             90275 non-null  float64
 9   decktypeid                    90275 non-null  string 
 10  finishedfloor1squarefeet      90275 non-null  int64  
 11  calculatedfinishedsquarefeet  90275 non-null  int64  
 12  finishedsquarefeet12          90275 non-null  int64  
 13  f

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,year_data,logerror,transactiondate
0,17073783,NA,NA,-1,2.5,3,NA,NA,2.5,NA,...,191811,2015,76724,2015.06,False,-1,61110022003007,2016,0.0953,2016-01-27
1,17088994,NA,NA,-1,1.0,2,NA,NA,1.0,NA,...,239679,2015,95870,2581.30,False,-1,61110015031002,2016,0.0198,2016-03-30
2,17100444,NA,NA,-1,2.0,3,NA,NA,2.0,NA,...,47853,2015,14234,591.64,False,-1,61110007011007,2016,0.0060,2016-05-27
3,17102429,NA,NA,-1,1.5,2,NA,NA,1.5,NA,...,62914,2015,17305,682.78,False,-1,61110008002013,2016,-0.0566,2016-06-07
4,17109604,NA,NA,-1,2.5,4,NA,NA,2.5,NA,...,554000,2015,277000,5886.92,False,-1,61110014021007,2016,0.0573,2016-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,11876798,1,NA,-1,3.0,4,NA,4,3.0,NA,...,596082,2015,207500,7335.81,False,-1,60372011202001,2016,0.0276,2016-04-15
90271,12808516,NA,NA,-1,2.0,3,NA,7,2.0,NA,...,178408,2015,92199,2441.74,False,-1,60374071013004,2016,-0.1188,2016-06-07
90272,12010248,1,NA,-1,2.0,2,NA,4,2.0,NA,...,420000,2015,291000,5070.41,False,-1,60371905102000,2016,0.0843,2016-06-30
90273,14310905,NA,NA,-1,2.5,3,NA,NA,2.5,NA,...,1215816,2015,826342,12508.30,False,-1,NA,2016,-0.0050,2016-03-31


In [14]:
# https://www.theclickreader.com/random-forest-regression/
from sklearn.preprocessing import OneHotEncoder # create one hot for strings
from sklearn.model_selection import train_test_split # splitting the data
from sklearn.metrics import mean_squared_error # cost function
from sklearn.ensemble import RandomForestRegressor # call the regressor model
from sklearn.model_selection import cross_val_score

def seperate_string(df):
    colnms_string = []
    colnms_nostring = []
    for idx, value in enumerate(df.dtypes.items()):
        if value[1] == "string":
            colnms_string.append(value[0])
        else:
            colnms_nostring.append(value[0])

    df_string = df[colnms_string]
    df_nostring = df[colnms_nostring]

    return (df_nostring, df_string)

dropcol = ['parcelid', 'censustractandblock'] # remove id 
dropcol.extend(['logerror', 'transactiondate']) # remove prediction target

df_pre = df_2016.drop(dropcol, axis = 1)
df_nostring, df_string = seperate_string(df_pre)
# df_string.info()
# printunique(df_string)

enc = OneHotEncoder(handle_unknown='ignore')
print(df_string.columns)

enc.fit(df_string)
dfarray = enc.transform(df_string).toarray()
dfcolnms = enc.get_feature_names_out(df_string.columns)

enc_df_string = pd.DataFrame(dfarray, columns = dfcolnms)

X = pd.concat([df_nostring, enc_df_string], axis = 1)
X.info()
y = df_2016['logerror']

Index(['airconditioningtypeid', 'architecturalstyletypeid',
       'buildingclasstypeid', 'buildingqualitytypeid', 'decktypeid', 'fips',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'regionidzip', 'storytypeid',
       'typeconstructiontypeid'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Columns: 2559 entries, basementsqft to typeconstructiontypeid_NA
dtypes: bool(3), float64(2519), int64(37)
memory usage: 1.7 GB


In [15]:
display(X)

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,regionidzip_97330,regionidzip_97331,regionidzip_97344,regionidzip_NA,storytypeid_7,storytypeid_NA,typeconstructiontypeid_13,typeconstructiontypeid_4,typeconstructiontypeid_6,typeconstructiontypeid_NA
0,-1,2.5,3,2.5,548,1264,1264,-1,-1,548,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,-1,1.0,2,1.0,777,777,777,-1,-1,777,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-1,2.0,3,2.0,1101,1101,1101,-1,-1,1101,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-1,1.5,2,1.5,1554,1554,1554,-1,-1,1554,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-1,2.5,4,2.5,1305,2415,2415,-1,-1,1305,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,-1,3.0,4,3.0,-1,2068,2068,-1,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
90271,-1,2.0,3,2.0,-1,1352,1352,-1,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
90272,-1,2.0,2,2.0,-1,860,860,-1,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
90273,-1,2.5,3,2.5,-1,2268,2268,-1,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [16]:
split_randonstate = 1947
model_randonstate = 1949
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = split_randonstate)
model = RandomForestRegressor(n_estimators = 20, max_depth = 10, oob_score = True, n_jobs = 6, random_state = model_randonstate, verbose = 2)
model.fit(x_train, y_train)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:   33.9s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:   33.9s finished
c:\Users\michaelshih\Anaconda3\envs\data2023\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestRegressor(max_depth=10, n_estimators=20, n_jobs=6, oob_score=True,
                      random_state=1949, verbose=2)

In [19]:
y_pred = model.predict(x_test)
# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  0.158


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    0.0s finished


In [6]:
model = RandomForestRegressor(n_estimators = 1000, max_depth = 20, oob_score = True, n_jobs = 8, verbose = 2)
score = cross_val_score(model, X, y, cv=5, verbose = 2, scoring = 'neg_mean_squared_error')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 13.9min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 67.4min


In [ ]:
print("Mean score of %0.2f with a standard deviation of %0.2f" % (score.mean(), score.std()))
print(score)

NameError: name 'scores' is not defined

In [8]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',